# Calculate AMOC #

In [10]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
sys.path.append('/home/563/sc1326/repos/cdrmip_extremes')
from cdrmip_extremes.configs import data_dir, models, expts
from cdrmip_extremes import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
client = Client(memory_limit=None,threads_per_worker=1,n_workers=28)

In [8]:
msftmz_dir = os.path.join(data,'raw/msftmz-msftyz')
msftmz = {model:{} for model in models}
for model in models:
    for expt in expts:
        path = os.path.join(msftmz_dir,expt,model,'*.nc')
        msftmz[model][expt] = xr.open_mfdataset(path,use_cftime=True)

In [9]:
msftmz

{'ACCESS-ESM1-5': {'1pctCO2': <xarray.Dataset> Size: 334MB
  Dimensions:    (time: 1800, bnds: 2, lev: 50, lat: 300, basin: 3)
  Coordinates:
    * time       (time) object 14kB 0101-01-16 12:00:00 ... 0250-12-16 12:00:00
      sector     (basin) |S21 63B dask.array<chunksize=(3,), meta=np.ndarray>
    * lev        (lev) float64 400B 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
    * lat        (lat) float64 2kB -77.75 -77.51 -77.26 -77.01 ... 89.1 89.55 90.0
  Dimensions without coordinates: bnds, basin
  Data variables:
      time_bnds  (time, bnds) object 29kB dask.array<chunksize=(1, 2), meta=np.ndarray>
      lev_bnds   (time, lev, bnds) float64 1MB dask.array<chunksize=(120, 50, 2), meta=np.ndarray>
      lat_bnds   (time, lat, bnds) float64 9MB dask.array<chunksize=(120, 300, 2), meta=np.ndarray>
      msftmz     (time, basin, lev, lat) float32 324MB dask.array<chunksize=(1, 3, 50, 300), meta=np.ndarray>
  Attributes: (12/46)
      Conventions:            CF-1.7 CMIP-6.2
      activity

In [11]:
concat = {}
for model in models:
    ds_up = msftmz[model]['1pctCO2']
    ds_down = msftmz[model]['1pctCO2-cdr']
    concat[model] = utils.concat_branches(ds_up,ds_down)

## Check model coordinates ##

In [33]:
amoc_configs = {model:{} for model in models}
for model in models:
    print(model)
    coords = list(concat[model].coords)
    lat_coord = [coord for coord in coords if coord in ['lat','rlat','y','j-mean']][0]
    variables = list(concat[model].variables)
    print(lat_coord)
    amoc_configs[model]['coord'] = lat_coord
    amoc_configs[model]['var'] = [variable for variable in variables if variable in ['msftmz','msftyz']][0]

ACCESS-ESM1-5
lat
CanESM5
lat
CESM2
lat
CNRM-ESM2-1
j-mean
GFDL-ESM4
y
MIROC-ES2L
lat
NorESM2-LM
lat
UKESM1-0-LL
rlat


In [23]:
def calc_amoc(str_func, basin_no, coord, lat):
    
    # isolate latitude of 26.5N in atlantic ocean
    str_func_26N = str_func.sel(
        basin=basin_no,
        drop=True
    ).sel(
        **{coord:lat},
        method='nearest',
        drop=True
    )
    
    # identify depth with maximum stream function
    # and select only these depths
    
    max_lev_idx = str_func_26N.idxmax(dim='lev')
    amoc_26N = str_func_26N.sel(lev=max_lev_idx,drop=True) / 1.025e9
    
    return amoc_26N

In [34]:
amoc_26N_dict = {}
for model in models:
    str_func = concat[model]
    # CNRM-ESM1-1 has different basin and latitude coordinates
    if model == 'CNRM-ESM2-1':
        basin_no = 1
        lat = 190.283
    else:
        basin_no = 0
        lat=26.5
    coord = amoc_configs[model]['coord']
    variable = amoc_configs[model]['var']
    print(coord)
    amoc_26N_dict[model] = calc_amoc(str_func[variable],basin_no,coord,lat)

lat
lat
lat
j-mean
y
lat
lat
rlat


In [35]:
### Save ###
save_dir = os.path.join(data_dir,'processed/amoc')
for model, ds in amoc_26N_dict.items():
    path = os.path.join(
        save_dir,
        f"{model}_amoc_26N.nc"
    )
    # ds.to_netcdf(path)

### and for piControl ###

In [37]:
amoc_pi_dict = {}
for model in models:

    str_func = msftmz[model]['piControl']
    # CNRM-ESM1-1 has different basin and latitude coordinates
    if model == 'CNRM-ESM2-1':
        basin_no = 1
        lat = 190.283
    else:
        basin_no = 0
        lat=26.5
    coord = amoc_configs[model]['coord']
    variable = amoc_configs[model]['var']
    print(coord)
    amoc_pi_dict[model] = calc_amoc(str_func[variable],basin_no,coord,lat)

lat
lat
lat
j-mean
y
lat
lat
rlat


In [ ]:
### Save ###
save_dir = os.path.join(data_dir,'processed/amoc')
for model, ds in amoc_pi_dict.items():
    path = os.path.join(
        save_dir,
        f"{model}_amoc_26N_piControl.nc"
    )
    # ds.to_netcdf(path)